In [ ]:
import matplotlib.pyplot as plt
from matplotlib import pylab as py
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
plt.rcParams['figure.figsize'] = 8,8

from scipy.ndimage.interpolation import zoom
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
import numpy as np
import os
import matplotlib.gridspec as gridspec

In [8]:
model = VGG16(include_top=True, weights='imagenet',input_shape=(224,224,3))
model.summary()

In [ ]:
def grad_cam_plus(input_model, img, layer_name,H=224,W=224):
    cls = np.argmax(input_model.predict(img))
    y_c = input_model.output[0, cls]
    conv_output = input_model.get_layer(layer_name).output
    grads = K.gradients(y_c, conv_output)[0]

    first = K.exp(y_c)*grads
    second = K.exp(y_c)*grads*grads
    third = K.exp(y_c)*grads*grads*grads

    gradient_function = K.function([input_model.input], [y_c,first,second,third, conv_output, grads])
    y_c, conv_first_grad, conv_second_grad,conv_third_grad, conv_output, grads_val = gradient_function([img])
    global_sum = np.sum(conv_output[0].reshape((-1,conv_first_grad[0].shape[2])), axis=0)

    alpha_num = conv_second_grad[0]
    alpha_denom = conv_second_grad[0]*2.0 + conv_third_grad[0]*global_sum.reshape((1,1,conv_first_grad[0].shape[2]))
    alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, np.ones(alpha_denom.shape))
    alphas = alpha_num/alpha_denom

    weights = np.maximum(conv_first_grad[0], 0.0)

    alpha_normalization_constant = np.sum(np.sum(alphas, axis=0),axis=0)

    alphas /= alpha_normalization_constant.reshape((1,1,conv_first_grad[0].shape[2]))

    deep_linearization_weights = np.sum((weights*alphas).reshape((-1,conv_first_grad[0].shape[2])),axis=0)
    grad_CAM_map = np.sum(deep_linearization_weights*conv_output[0], axis=2)

    cam = np.maximum(grad_CAM_map, 0)
    cam = zoom(cam,H/cam.shape[0])
    cam = cam / np.max(cam) # scale 0 to 1.0    
    cam = resize(cam, (H,W))
    print(cam.shape)
    return cam

In [ ]:
pathf="n03788195_154.JPEG"
orig_img = np.array(load_img(pathf,target_size=(224,224)),dtype=np.uint8)
img = np.array(load_img(pathf,target_size=(224,224)),dtype=np.float64)
img = np.expand_dims(img,axis=0)
img = preprocess_input(img)
predictions = model.predict(img)
top_n = 5
top = decode_predictions(predictions, top=top_n)[0]
cls = np.argsort(predictions[0])[-top_n:][::-1]
layers=["block1_conv1","block1_conv2","block2_conv1","block2_conv2","block3_conv1","block3_conv2","block3_conv3",
       "block4_conv1","block4_conv2","block4_conv3","block5_conv1","block5_conv2","block5_conv3"]

lis=[]

    for k in range(len(layers)):

        gradcamplus=gradcamutils.grad_cam_plus(model,img,layer_name=layers[k])
        lis.append(gradcamplus)
    fig1, ax1 = plt.subplots(nrows=2,ncols=7,figsize=(20,6))
    plt.subplot(2,7,1)
    plt.imshow(orig_img)
    plt.title("input image")
    plt.subplot(2,7,2)
    plt.imshow(orig_img)
    plt.imshow(lis[0],alpha=0.8,cmap="jet")
    plt.title("block1_conv1")
    plt.subplot(2,7,3)
    plt.imshow(orig_img)
    plt.imshow(lis[1],alpha=0.8,cmap="jet")
    plt.title("block1_conv2")
    plt.subplot(2,7,4)
    plt.imshow(orig_img)
    plt.imshow(lis[2],alpha=0.8,cmap="jet")
    plt.title("block2_conv1")
    plt.subplot(2,7,5)
    plt.imshow(orig_img)
    plt.imshow(lis[3],alpha=0.8,cmap="jet")
    plt.title("block2_conv2")
    plt.subplot(2,7,6)
    plt.imshow(orig_img)
    plt.imshow(lis[4],alpha=0.8,cmap="jet")
    plt.title("block3_conv1")
    plt.subplot(2,7,7)
    plt.imshow(orig_img)
    plt.imshow(lis[5],alpha=0.8,cmap="jet")
    plt.title("block3_conv2")
    plt.subplot(2,7,8)
    plt.imshow(orig_img)
    plt.imshow(lis[6],alpha=0.8,cmap="jet")
    plt.title("block3_conv3")
    plt.subplot(2,7,9)
    plt.imshow(orig_img)
    plt.imshow(lis[7],alpha=0.8,cmap="jet")
    plt.title("block4_conv1")
    plt.subplot(2,7,10)
    plt.imshow(orig_img)
    plt.imshow(lis[8],alpha=0.8,cmap="jet")
    plt.title("block4_conv2")
    plt.subplot(2,7,11)
    plt.imshow(orig_img)
    plt.imshow(lis[9],alpha=0.8,cmap="jet")
    plt.title("block4_conv3")
    plt.subplot(2,7,12)
    plt.imshow(orig_img)
    plt.imshow(lis[10],alpha=0.8,cmap="jet")
    plt.title("block5_conv1")
    plt.subplot(2,7,13)
    plt.imshow(orig_img)
    plt.imshow(lis[11],alpha=0.8,cmap="jet")
    plt.title("block5_conv2")
    plt.subplot(2,7,14)
    plt.imshow(orig_img)
    plt.imshow(lis[12],alpha=0.8,cmap="jet")
    plt.title("block5_conv3")
    fig1.subplots_adjust(wspace=0.3,hspace=0.1)
    plt.show()
        



  0%|                                                                                           | 0/10 [00:00<?, ?it/s]